<a href="https://colab.research.google.com/github/hscrown/bigdata/blob/main/05_2_%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C_%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
## validation set 을 만들지 않으면 결국 test set에 과대적합한 모델을 만들 수 밖에 없다
## train, val ,  test 셋을 나누고 val세트에 맞추기. test는 딱 한 번 테스트만

# 데이터 임포트
import pandas as pd

wine = pd.read_csv('http://bit.ly/wine_csv_data')

data = wine.drop(labels=['class'],axis=1).to_numpy()
target = wine['class'].to_numpy()

In [4]:
## train, val, test set 나누는 방법

## train, test 셋으로 나누고 train 셋을 다시 train, val로 나눔
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [5]:
## 결정 트리로 모델 훈련
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [6]:
## 교차 검증:데이터를 여러 개의 부분으로 나누어 모델을 평가하고 성능을 일반화하는 기법
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target) # cv=5(폴드갯수)가 기본값
print(scores) # 각 폴드에 대한 성능 지표 반환

# fit_time: 모델 훈련에 걸린 시간.
# score_time: 모델 평가에 걸린 시간.
# test_score: 각 폴드에서의 모델 정확도.

{'fit_time': array([0.01384449, 0.0351007 , 0.02723217, 0.03702569, 0.01191521]), 'score_time': array([0.00194097, 0.00932145, 0.01050544, 0.0018816 , 0.00205278]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [7]:
## 모든 폴드에 대한 평균 스코어
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [9]:
from sklearn.model_selection import StratifiedKFold
# StratifiedKFold를 기본 설정으로 사용하여 cross_validate를 호출
# 기본적으로 폴드 수는 5개, 데이터 섞기와 난수 시드는 설정되지 않음
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())  # 기본 StratifiedKFold 사용
print(np.mean(scores['test_score']))  # 각 폴드에서의 테스트 점수 평균 출력


# StratifiedKFold 객체를 생성하면서 폴드 수, 데이터 섞기 및 난수 시드를 설정
# n_splits=10: 폴드 수를 10개로 설정하여 더 많은 데이터가 테스트에 사용되도록 함
# shuffle=True: 데이터를 무작위로 섞어 순서에 따른 영향을 방지
# random_state=42: 난수 시드를 고정하여 데이터 분할을 재현 가능하게 만듦
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# cross_validate 함수에서 이 StratifiedKFold 객체를 사용하여 교차 검증 수행
# 사용자 지정 StratifiedKFold 사용: 10개 폴드, 데이터 섞기, 난수 시드 고정
scores = cross_validate(dt, train_input, train_target, cv=splitter)  # 사용자 지정 StratifiedKFold 사용
print(np.mean(scores['test_score']))  # 각 폴드에서의 테스트 점수 평균 출력


0.855300214703487
0.8574181117533719


### 하이퍼파라미터 튜닝
- 그리드 서치
- 랜덤 서치

In [10]:
from sklearn.model_selection import GridSearchCV
import numpy as np

# 파라미터 후보 설정
# param = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
param = {
    'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),  # min_impurity_decrease 값을 0.0001부터 0.001까지 테스트
    'max_depth': range(5, 20, 1),  # max_depth 값을 5부터 19까지 테스트
    'min_samples_split': range(1, 100, 10)  # min_samples_split 값을 1부터 99까지 10단위로 테스트
}

# GridSearchCV 객체 생성
# DecisionTreeClassifier를 모델로 사용하고, param을 통해 하이퍼파라미터 튜닝
# n_jobs=-1: 가능한 모든 CPU 코어를 사용하여 병렬 처리
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), param, n_jobs=-1)

# train_input과 train_target을 사용하여 그리드 서치 수행
gs.fit(train_input, train_target)

# best_estimator_ 속성으로 최적의 모델을 가져옴
dt = gs.best_estimator_

# 최적 모델의 정확도를 훈련 데이터에서 출력
print(dt.score(train_input, train_target))

# 최적의 파라미터를 출력 (best_params_는 그리드 서치에서 선택한 최적 파라미터 값)
print(gs.best_params_)

# 각 하이퍼파라미터 값에 대해 교차 검증 결과의 평균 점수 출력
print(gs.cv_results_['mean_test_score'])

# 평균 테스트 점수가 가장 높은 인덱스를 찾아 최적의 파라미터 값 출력
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])


0.8930152010775447
{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 11}
[       nan 0.85799604 0.85799604 ... 0.86126601 0.86165063 0.86357629]
{'max_depth': 5, 'min_impurity_decrease': 0.0001, 'min_samples_split': 1}


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
675 fits failed out of a total of 6750.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
675 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/uti

In [12]:
### 랜덤서치: 매개변수 값이 수치여서 값이 범위나 간격을 미리 정하기 어려울 때, 많은 매개 변수 조건이 있어 그리드 서치가 오래걸릴때
## 매개변수 값의 목록을 전달(X)
## 매개변수를 샘플링 할 수 있는 확률 분포 객체를 전달

# 사이파이 클래스에서 2개의 확률 분포 클래스를 임포트 하기
from scipy.stats import uniform, randint # 주어진 범위에서 고르게 값을 뽑는 클래스들. 균등분포에서 샘플링 한다. 정수분포를 뽑음(randint), 실숫값을 뽑음(uniform)

# 0~10사이의 정수 추출

# 0~10사이의 범위를 같는 randint 객체 선언
rgen = randint(0, 10)
# 10개의 숫자를 샘플링
rgen.rvs(rgen.rvs(10))

array([], shape=(7, 8, 1, 6, 3, 5, 0, 9, 2, 3), dtype=int64)

In [14]:
# 1000개의 숫자를 샘플링
r1000 = rgen.rvs(1000)
np.unique(r1000, return_counts=True) # 0부터 10까지 각 숫자의 갯수

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([105,  85,  92,  89,  92, 104, 126, 106,  90, 111]))

In [15]:
# 0~1사이의 10개의 실수 추출
ugen = uniform(0,1)
ugen.rvs(10)

array([0.77302787, 0.75890977, 0.84269189, 0.39430248, 0.80878599,
       0.55156649, 0.6106969 , 0.60821406, 0.41799467, 0.55495346])

In [16]:
# 랜덤 서치
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a7fd9ea2560>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7a7fda1f3b50>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a7fda1f1d80>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a7fd9ea2170>},
                   random_state=42)

In [17]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884
0.86


In [19]:
## splitter = 'random' 으로 설정한것과 , 랜덤 서치로 파라미터 서치한 것 비교
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target)) # 랜덤 서치보다 정확도가 낮음

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
